In [1]:
!pip install transformers

     |████████████████████████████████| 1.8MB 16.0MB/s 
     |████████████████████████████████| 890kB 56.8MB/s 
     |████████████████████████████████| 2.9MB 51.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=22fa5c488a74afac34f60bd4073c362c34aeb81c2f06ec0aec133235e4fdfd7c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import BertTokenizer, TFBertForSequenceClassification, InputExample, InputFeatures

In [3]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [5]:
import tensorflow as tf
import pandas as pd

In [6]:
URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz",
                                  origin=URL,
                                  untar = True,
                                  cache_dir = '.',
                                  cache_subdir = '')

84131840/84125825 [==============================] - 4s 0us/step


In [7]:
import os
import shutil

In [8]:
main_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

train_dir = os.path.join(main_dir, 'train')

remove_dir = os.path.join(train_dir, 'unsup')

shutil.rmtree(remove_dir)

print(os.listdir(train_dir))

['urls_pos.txt', 'labeledBow.feat', 'urls_neg.txt', 'pos', 'unsupBow.feat', 'neg', 'urls_unsup.txt']


In [9]:
train = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train', batch_size=30000, validation_split=0.2, subset='training', seed=123)

test = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train', batch_size=30000, validation_split=0.2, subset='validation', seed=123)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [13]:
for i in train.take(1):
  train_feat = i[0].numpy()
  train_lab = i[1].numpy()

train = pd.DataFrame([train_feat, train_lab]).T
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
train['DATA_COLUMN'] = train['DATA_COLUMN'].str.decode("utf-8")


AssertionError: ignored

In [16]:
train['DATA_COLUMN'].head()

0    Canadian director Vincenzo Natali took the art...
1    I gave this film 10 not because it is a superb...
2    I admit to being somewhat jaded about the movi...
3    For a long time, 'The Menagerie' was my favori...
4    A truly frightening film. Feels as if it were ...
Name: DATA_COLUMN, dtype: object

In [11]:
for i in test.take(1):
  test_feat = i[0].numpy()
  test_lab = i[1].numpy()

test = pd.DataFrame([test_feat, test_lab]).T
test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
test['DATA_COLUMN'] = test['DATA_COLUMN'].str.decode("utf-8")
test.head()

,DATA_COLUMN,LABEL_COLUMN
0,I can't believe that so much talent can be was...,0
1,This movie blows - let's get that straight rig...,0
2,"The saddest thing about this ""tribute"" is that...",0
3,I'm only rating this film as a 3 out of pity b...,0
4,Something surprised me about this movie - it w...,1


In [ ]:
InputExample(guid=None,
             text_a = "Hello, World",
             text_b = None,
             label = 1)

InputExample(guid=None, text_a='Hello, World', text_b=None, label=1)

In [ ]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN):
  train_InputExamples = train.apply(lambda x: InputExample(guid=None,
                                                           text_a = x[DATA_COLUMN],
                                                           text_b = None,
                                                           label = x[LABEL_COLUMN]),axis=1)
 
  validation_InputExamples = test.apply(lambda x: InputExample(guid=None,
                                                           text_a = x[DATA_COLUMN],
                                                           text_b = None,
                                                           label = x[LABEL_COLUMN]),axis=1)
  
  return train_InputExamples, validation_InputExamples


In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train,test,'DATA_COLUMN','LABEL_COLUMN')

In [ ]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] 

    for e in examples:
        
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, 
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, 
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5,epsilon=1e-08,clipnorm = 1.0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

In [ ]:
model.fit(train_data, epochs=5, validation_data=validation_data)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fac67e20d90> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fac67e20d90> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


   1250/Unknown - 1134s 877ms/step - loss: 0.3711 - accuracy: 0.8242

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1250/1250 [==============================] - 1180s 913ms/step - loss: 0.3710 - accuracy: 0.8243 - val_loss: 0.3090 - val_accuracy: 0.8784
Epoch 2/5
1250/1250 [==============================] - 1141s 913ms/step - loss: 0.0973 - accuracy: 0.9653 - val_loss: 0.4873 - val_accuracy: 0.8778
Epoch 3/5
1250/1250 [==============================] - 1141s 913ms/step - loss: 0.0344 - accuracy: 0.9890 - val_loss: 0.5760 - val_accuracy: 0.8814
Epoch 4/5
1250/1250 [==============================] - 1141s 913ms/step - loss: 0.0231 - accuracy: 0.9928 - val_loss: 0.6525 - val_accuracy: 0.8748
Epoch 5/5
1250/1250 [==============================] - 1140s 912ms/step - loss: 0.0133 - accuracy: 0.9962 - val_loss: 0.6324 - val_accuracy: 0.8740


In [ ]:
model.save_weights('Sentiment.h5')

In [ ]:
pred_sentences = ['Movie is good but not excellent']

In [ ]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0],axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions,axis=1) 
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])

Movie is good but not excellent : 
 Positive
